# Multi-Omics Analysis for LUSC Survival Prediction

Bla



## Imports and helper functions

In [54]:
import os,io,re,json,glob,tarfile,requests

import numpy as np
import pandas as pd

from pypgatk.cgenomes.cbioportal_downloader import CbioPortalDownloadService

In [95]:
def untar_study(study, output_directory, fmt=".tar.gz"):
    file = tarfile.open(
        os.path.join(
            output_directory,
            "{study}{format}".format(study=study, format=fmt)
            )
        )
    file.extractall(output_directory)
    file.close()

In [ ]:
def download_cbioportal_study(
        config_file,
        study,
        output_directory = "./data",
        list_studies = False,
        multithreading = True,
        **kwargs
        ):
    pipeline_arguments = {
        CbioPortalDownloadService.CONFIG_OUTPUT_DIRECTORY: output_directory,
        CbioPortalDownloadService.CONFIG_LIST_STUDIES: list_studies,
        CbioPortalDownloadService.CONFIG_MULTITHREADING: multithreading,
        **kwargs,
    } 
    cbioportal_downloader_service = CbioPortalDownloadService(config_file, pipeline_arguments)
    cbioportal_downloader_service.download_study(study)

In [70]:
def get_mirna_files(
        project_id="TCGA-LUSC",
        maxfiles=10000
        ):

    cases_endpt = "https://api.gdc.cancer.gov/files"
    data_endpt = "https://api.gdc.cancer.gov/data"

    # Retrieve associated file names
    filters = {
        "op": "and",
        "content":[
            {"op": "=",
            "content":{
                "field": "cases.project.project_id",
                "value": ["TCGA-LUSC"]
                }
            },
            {"op": "=",
            "content":{
                "field": "files.experimental_strategy",
                "value": ["miRNA-Seq"]
                }
            },
            {"op": "=",
            "content":{
                "field": "files.data_category",
                "value": ["Transcriptome Profiling"]
                }
            },
            {"op": "=",
            "content":{
                "field": "files.data_type",
                "value": ["miRNA Expression Quantification"]
                }
            }
        ]
    }

    params = {
        "filters": json.dumps(filters),
        "fields": ",".join(["cases.submitter_id","file_name"]),
        "format": "TSV",
        "size": str(maxfiles)
    }

    response = requests.get(cases_endpt, params = params)
    files_df = pd.read_csv(io.StringIO(response.text), sep="\t")
    return files_df

def download_mirna_files(
        files_df,
        output_directory = "./data",
        project_id="TCGA-LUSC",
        maxfiles=10000
        ):
    
    params = {"ids": files_df["id"].tolist()}

    response = requests.post(data_endpt,
                            data = json.dumps(params),
                            headers={
                                "Content-Type": "application/json"
                                })

    response_head_cd = response.headers["Content-Disposition"]
    file_name = re.findall("filename=(.+)", response_head_cd)[0]
    with open(os.path.join(output_directory, file_name), "wb") as output_file:
        output_file.write(response.content)
    return file_name

In [ ]:
def untar_and_merge_mirna_files(
        files_df,
        file_name,
        output_directory = "./data",
        ):
    

## Data Acquisition and Pre-Processing

### Acquiring data from cBioPortal

In [16]:
cbioportal_config = "./config/cbioportal_config.yaml"
data_directory = "./data"
study_name = "lusc_tcga"

In [10]:
download_cbioportal_study(cbioportal_config, study_name, data_directory)

DEBUG:pypgatk.toolbox.general:The following study 'lusc_tcga' has been downloaded. 


In [13]:
untar_study(study_name, data_directory, fmt=".tar")

In [18]:
glob.glob()

['./data/lusc_tcga/data_RNA_Seq_v2_expression_median.txt',
 './data/lusc_tcga/data_bcr_clinical_data_patient.txt',
 './data/lusc_tcga/data_CNA.txt',
 './data/lusc_tcga/data_mRNA_median_all_sample_Zscores.txt',
 './data/lusc_tcga/data_linear_CNA.txt',
 './data/lusc_tcga/data_mRNA_median_Zscores.txt',
 './data/lusc_tcga/data_mutsig.txt',
 './data/lusc_tcga/data_expression_all_sample_Zscores.txt',
 './data/lusc_tcga/data_bcr_clinical_data_sample.txt',
 './data/lusc_tcga/data_expression.txt',
 './data/lusc_tcga/data_methylation_hm450.txt',
 './data/lusc_tcga/data_mutations_extended.txt',
 './data/lusc_tcga/data_RNA_Seq_v2_mRNA_median_all_sample_Zscores.txt',
 './data/lusc_tcga/data_expression_Zscores.txt',
 './data/lusc_tcga/data_rppa_Zscores.txt',
 './data/lusc_tcga/data_rppa.txt',
 './data/lusc_tcga/data_mutations_mskcc.txt',
 './data/lusc_tcga/data_gistic_genes_del.txt',
 './data/lusc_tcga/data_expression_median.txt',
 './data/lusc_tcga/data_RNA_Seq_v2_mRNA_median_Zscores.txt',
 './data

In [48]:
RNAseq = pd.read_csv(os.path.join(data_directory, study_name, "data_RNA_Seq_v2_expression_median.txt"), comment="#", sep="\t").set_index("Hugo_Symbol").drop(columns="Entrez_Gene_Id")
linearCNA = pd.read_csv(os.path.join(data_directory, study_name, "data_linear_CNA.txt"), comment="#", sep="\t").set_index("Hugo_Symbol").drop(columns="Entrez_Gene_Id")
methylation_hm450 = pd.read_csv(os.path.join(data_directory, study_name, "data_methylation_hm450.txt"), comment="#", sep="\t").set_index("Hugo_Symbol").drop(columns="Entrez_Gene_Id")

In [49]:
RNAseq_t = RNAseq.transpose()
linearCNA_t = linearCNA.transpose()
methylation_hm450_t = methylation_hm450.transpose()

In [33]:
RNAseq.describe()

,TCGA-18-3406-01,TCGA-18-3407-01,TCGA-18-3408-01,TCGA-18-3409-01,TCGA-18-3410-01,TCGA-18-3411-01,TCGA-18-3412-01,TCGA-18-3414-01,TCGA-18-3415-01,TCGA-18-3416-01,...,TCGA-NK-A5CX-01,TCGA-NK-A5D1-01,TCGA-NK-A7XE-01,TCGA-O2-A52N-01,TCGA-O2-A52Q-01,TCGA-O2-A52S-01,TCGA-O2-A52V-01,TCGA-O2-A52W-01,TCGA-O2-A5IB-01,TCGA-XC-AA0X-01
count,20531.000000,20531.000000,20531.000000,20531.000000,20531.000000,20531.000000,20531.000000,20531.000000,20531.000000,20531.000000,...,20531.000000,20531.000000,20531.000000,20531.000000,20531.000000,20531.000000,20531.000000,20531.000000,20531.000000,20531.000000
mean,1395.257560,978.985025,969.610105,999.404257,939.557529,1065.102322,1094.114204,900.748332,1066.275696,923.566782,...,1005.773082,882.988249,965.729134,996.180572,991.993145,917.191129,963.178447,955.965443,849.944151,972.787785
std,10645.015369,4066.571581,3437.254287,4937.694979,3397.076921,5854.093447,5352.920394,2834.714177,4757.822855,4128.842868,...,4191.071573,2710.419548,3968.143264,3964.704669,4156.316787,2906.830044,3339.015162,3842.677629,2404.802071,3752.501889
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.661400,5.708800,4.152500,8.717900,7.483800,4.425200,5.086100,8.861000,5.202300,3.347400,...,6.357700,13.434600,5.056900,7.103800,3.976400,6.272750,7.143600,5.910800,4.635300,13.049700
50%,202.362200,199.334000,151.861200,224.102600,219.000900,179.422700,187.793400,225.586100,180.346800,180.759900,...,210.783700,240.642500,180.151700,204.918000,194.346100,203.642400,214.728400,195.056400,180.777400,237.280400
75%,820.472400,837.095600,794.305200,840.769200,866.747250,823.695050,824.334900,860.439400,825.722550,813.419800,...,834.331850,861.273350,815.739550,851.784150,820.627500,858.845200,858.913800,837.487900,847.310950,879.551100
max,657318.897600,232486.441500,135715.260300,305587.692300,155924.692800,607082.369500,373877.543000,130156.544200,228073.410400,346779.027900,...,201507.274700,118961.448600,211838.179500,256240.437200,186599.068000,127838.990100,136087.824400,176318.108500,114636.581500,223280.699500


In [36]:
RNAseq_t.describe()

Hugo_Symbol,LOC100130426,UBE2Q2P3,UBE2Q2P3,HMGB1P1,TIMM23,MOXD2,LOC155060,RNU12-2P,SSX9,LOC317712,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,FLJ10821,ZZZ3,TPTEP1,AKR1C6P
count,501.000000,501.000000,501.000000,501.000000,501.000000,501.0,501.000000,501.000000,501.000000,501.0,...,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000
mean,0.012149,21.606445,18.214324,169.930685,1033.474601,0.0,182.181532,0.493782,0.048175,0.0,...,49.338248,348.530790,1426.308160,61.217290,674.079946,4497.794075,990.893787,786.803342,162.075459,1.318383
std,0.066801,18.572677,14.447628,70.732988,446.680819,0.0,156.118382,0.698343,0.592934,0.0,...,21.910006,136.664958,557.774866,77.333066,201.962526,2252.989232,404.887158,268.549282,765.379422,6.335176
min,0.000000,0.000000,0.000000,50.018800,407.969600,0.0,3.371400,0.000000,0.000000,0.0,...,4.214300,63.903300,314.870600,0.000000,188.550800,526.175300,162.310000,271.528300,0.327500,0.000000
25%,0.000000,8.548500,8.568800,121.551700,746.829000,0.0,88.028500,0.000000,0.000000,0.0,...,34.243900,257.269300,1043.486500,6.950000,538.119500,2873.575700,700.203700,595.955400,7.503800,0.000000
50%,0.000000,17.170700,14.413200,154.934300,935.168000,0.0,141.972100,0.353600,0.000000,0.0,...,45.601300,329.375200,1344.295000,30.807700,650.977300,4072.087900,948.110800,739.910300,16.653200,0.000000
75%,0.000000,28.560000,22.793300,199.713400,1185.133600,0.0,230.152900,0.719200,0.000000,0.0,...,61.786200,408.026800,1694.603900,86.180800,778.177500,5497.894000,1239.779900,907.422900,68.526600,0.516800
max,0.582500,125.101500,85.153600,473.632600,4533.721600,0.0,1262.278600,8.545300,11.252200,0.0,...,164.110000,1220.662300,4782.939800,594.511400,2181.599400,17037.809600,2665.348200,2236.807700,11065.273000,115.485500


In [42]:
linearCNA.describe()

,TCGA-18-3406-01,TCGA-18-3407-01,TCGA-18-3408-01,TCGA-18-3409-01,TCGA-18-3410-01,TCGA-18-3411-01,TCGA-18-3412-01,TCGA-18-3414-01,TCGA-18-3415-01,TCGA-18-3416-01,...,TCGA-NK-A5CX-01,TCGA-NK-A5D1-01,TCGA-NK-A7XE-01,TCGA-O2-A52N-01,TCGA-O2-A52Q-01,TCGA-O2-A52S-01,TCGA-O2-A52V-01,TCGA-O2-A52W-01,TCGA-O2-A5IB-01,TCGA-XC-AA0X-01
count,24776.000000,24776.000000,24776.000000,24776.000000,24776.000000,24776.000000,24776.000000,24776.000000,24776.000000,24776.000000,...,24776.00000,24776.000000,24776.000000,24776.000000,24776.000000,24776.000000,24776.000000,24776.000000,24776.000000,24776.000000
mean,-0.039174,0.044251,-0.031631,-0.002381,0.089617,0.104001,0.145466,0.108006,-0.034935,-0.013450,...,0.07353,-0.177821,0.283053,0.159393,-0.045359,-0.086362,-0.048340,0.082023,-0.038609,0.008012
std,0.333807,0.280860,0.453350,0.246527,0.530954,0.527749,0.720944,0.547131,0.374003,0.492359,...,0.27356,0.517342,0.412614,0.578000,0.365489,0.471900,0.240554,0.451727,0.684630,0.243126
min,-1.065000,-1.055000,-1.121000,-1.028000,-1.293000,-1.069000,-1.293000,-1.293000,-1.293000,-1.241000,...,-1.29300,-1.293000,-0.839000,-1.293000,-1.293000,-1.293000,-0.803000,-1.110000,-1.293000,-1.293000
25%,-0.231000,-0.085000,-0.118000,-0.178000,-0.099000,-0.350000,-0.259000,-0.107000,-0.347000,-0.382000,...,-0.10400,-0.453000,-0.016000,-0.137000,-0.391000,-0.553000,-0.281000,-0.320000,-0.670000,-0.134000
50%,-0.004000,0.000000,0.000000,-0.014500,-0.004000,0.000000,0.000000,0.000000,-0.009000,0.000000,...,0.00000,0.000000,-0.002000,0.000000,-0.001000,-0.042000,0.011000,0.000000,0.008000,-0.007000
75%,0.043000,0.184000,0.039000,0.110000,0.470000,0.381000,0.433000,0.318000,0.063000,0.348000,...,0.11600,0.032000,0.654000,0.341500,0.048000,0.081000,0.069000,0.297000,0.056000,0.057000
max,3.596000,3.025000,3.657000,1.404000,3.657000,3.503000,3.657000,3.657000,2.991000,3.657000,...,2.36700,3.657000,3.657000,3.657000,3.657000,3.657000,3.377000,3.657000,2.461000,2.487000


In [43]:
linearCNA_t.describe()

Hugo_Symbol,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,C1orf159,...,SMIM9,SNORA36A,SNORA56,TMLHE,VBP1,IL9R|ENSG00000124334.12,SPRY3|ENSG00000168939.6,VAMP7|ENSG00000124333.10,WASH6P|ENSG00000182484.10,WASIR1|ENSG00000185203.7
count,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,...,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000
mean,-0.113760,-0.113760,-0.113760,-0.113760,-0.113760,-0.113760,-0.113760,-0.113760,-0.113760,-0.113760,...,0.066888,0.066888,0.066888,0.063878,0.066888,0.057952,0.057952,0.057952,0.057952,0.057952
std,0.321659,0.321659,0.321659,0.321659,0.321659,0.321659,0.321659,0.321659,0.321659,0.321659,...,0.333737,0.333737,0.333737,0.335132,0.333737,0.330098,0.330098,0.330098,0.330098,0.330098
min,-0.947000,-0.947000,-0.947000,-0.947000,-0.947000,-0.947000,-0.947000,-0.947000,-0.947000,-0.947000,...,-0.924000,-0.924000,-0.924000,-0.924000,-0.924000,-0.924000,-0.924000,-0.924000,-0.924000,-0.924000
25%,-0.330000,-0.330000,-0.330000,-0.330000,-0.330000,-0.330000,-0.330000,-0.330000,-0.330000,-0.330000,...,-0.073000,-0.073000,-0.073000,-0.075000,-0.073000,-0.078000,-0.078000,-0.078000,-0.078000,-0.078000
50%,-0.051000,-0.051000,-0.051000,-0.051000,-0.051000,-0.051000,-0.051000,-0.051000,-0.051000,-0.051000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.029000,0.029000,0.029000,0.029000,0.029000,0.029000,0.029000,0.029000,0.029000,0.029000,...,0.102000,0.102000,0.102000,0.097000,0.102000,0.094000,0.094000,0.094000,0.094000,0.094000
max,1.864000,1.864000,1.864000,1.864000,1.864000,1.864000,1.864000,1.864000,1.864000,1.864000,...,2.957000,2.957000,2.957000,2.957000,2.957000,2.957000,2.957000,2.957000,2.957000,2.957000


In [50]:
methylation_hm450.describe()

,TCGA-18-3417-01,TCGA-18-4721-01,TCGA-18-5592-01,TCGA-18-5595-01,TCGA-21-5782-01,TCGA-21-5783-01,TCGA-21-5784-01,TCGA-21-5786-01,TCGA-21-5787-01,TCGA-21-A5DI-01,...,TCGA-NK-A5CX-01,TCGA-NK-A5D1-01,TCGA-NK-A7XE-01,TCGA-O2-A52N-01,TCGA-O2-A52Q-01,TCGA-O2-A52S-01,TCGA-O2-A52V-01,TCGA-O2-A52W-01,TCGA-O2-A5IB-01,TCGA-XC-AA0X-01
count,16711.000000,16707.000000,16713.000000,16702.000000,16682.000000,16711.000000,16701.000000,16713.000000,16704.000000,16712.000000,...,16714.000000,16714.000000,16656.000000,16714.000000,16714.000000,16712.000000,16703.000000,16714.000000,16714.000000,16645.000000
mean,0.393690,0.416031,0.393607,0.393502,0.365827,0.388327,0.381489,0.414131,0.410781,0.362539,...,0.376794,0.359473,0.398533,0.419731,0.422010,0.436773,0.395659,0.391381,0.431253,0.384579
std,0.334558,0.339004,0.353080,0.349458,0.314078,0.330363,0.326753,0.345327,0.347276,0.311027,...,0.318330,0.344871,0.333040,0.346830,0.342522,0.351518,0.317308,0.324605,0.374980,0.309475
min,0.006171,0.006163,0.007503,0.008699,0.006867,0.009924,0.006832,0.007810,0.004948,0.011064,...,0.009737,0.009606,0.008191,0.009999,0.010107,0.008514,0.009561,0.009764,0.009399,0.008940
25%,0.046647,0.052703,0.045554,0.049414,0.051388,0.055074,0.045439,0.047221,0.046039,0.049088,...,0.048666,0.042830,0.059029,0.052736,0.054215,0.062095,0.064349,0.048628,0.049036,0.070053
50%,0.360332,0.404073,0.271423,0.274912,0.318418,0.320411,0.337539,0.390913,0.371766,0.314717,...,0.340431,0.188277,0.350969,0.386036,0.402979,0.414660,0.372526,0.374720,0.327762,0.341022
75%,0.707354,0.750533,0.776245,0.769837,0.640374,0.715499,0.687892,0.761527,0.767297,0.634718,...,0.661482,0.732978,0.731709,0.779483,0.771868,0.803395,0.696271,0.689660,0.849688,0.671630
max,0.993903,0.993024,0.990796,0.991801,0.991611,0.988631,0.990710,0.991829,0.991063,0.988449,...,0.990107,0.989906,0.991606,0.988841,0.990946,0.990908,0.989270,0.989747,0.990209,0.990122


In [51]:
methylation_hm450_t.describe()

Hugo_Symbol,ITGAD,CHCHD4,C3orf16,C14orf181,MTMR7,RRS1,PTPRF,C6orf168,GFPT2,ROD1,...,OXCT1,PHACTR2,RABGEF1,CLDN12,PRKRIP1,TFEC,MKLN1,KIAA1432,AUH,ZNF189
count,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,370.000000,369.000000,...,357.000000,370.000000,370.000000,369.000000,370.000000,362.000000,369.000000,367.000000,370.000000,370.000000
mean,0.804234,0.043617,0.510268,0.059269,0.695712,0.184008,0.479846,0.287044,0.842836,0.186055,...,0.130784,0.049343,0.055219,0.103354,0.055194,0.112699,0.079302,0.112309,0.059124,0.089862
std,0.109520,0.009893,0.195670,0.018914,0.183735,0.076754,0.147810,0.128425,0.081286,0.098885,...,0.067522,0.030241,0.032143,0.046786,0.039331,0.059604,0.039424,0.050945,0.022574,0.031283
min,0.110923,0.024681,0.093391,0.025606,0.102060,0.035327,0.080879,0.042508,0.251573,0.045873,...,0.037197,0.020267,0.027681,0.019193,0.013810,0.027895,0.028862,0.037788,0.021724,0.039230
25%,0.778990,0.036818,0.361220,0.043680,0.594793,0.130825,0.375882,0.191747,0.814499,0.121528,...,0.080774,0.036554,0.041421,0.073254,0.026741,0.066168,0.055807,0.078110,0.044438,0.067334
50%,0.838831,0.042689,0.498606,0.056633,0.733047,0.173737,0.481256,0.281575,0.860373,0.165400,...,0.109542,0.042224,0.047963,0.089778,0.042097,0.089913,0.065502,0.098993,0.053259,0.082957
75%,0.867736,0.048741,0.652309,0.072397,0.848661,0.225336,0.571424,0.359517,0.895375,0.227407,...,0.160832,0.052360,0.060241,0.119816,0.074713,0.148710,0.086788,0.132033,0.066069,0.106344
max,0.924896,0.107466,0.920409,0.124612,0.942823,0.547342,0.931582,0.724943,0.949513,0.786247,...,0.377582,0.410086,0.534199,0.411113,0.325855,0.326553,0.256924,0.350641,0.201730,0.216217


### miRNA data from gdc.cancer.gov

In [73]:
mirna_files_df = get_mirna_files()
mirna_files_df

,cases.0.submitter_id,file_name,id
0,TCGA-22-1011,7df584d3-5b11-4987-8227-6d809e8305af.mirbase21...,07fa22d0-99b1-4364-9b93-d937005a2416
1,TCGA-56-A4ZK,4fca772a-da57-49ce-bc0f-6e0f0ddb50f2.mirbase21...,1b47db10-6bbd-454b-9ef0-0a2eba72f641
2,TCGA-63-A5MH,79c19963-dbac-4bbd-879d-80bf72211c62.mirbase21...,5dbae45f-de7d-4517-ad27-4da44d6b6c7a
3,TCGA-22-5482,ce46e1ee-09b5-40f9-9494-e02ffd8921e3.mirbase21...,89a26591-e6b8-4810-8092-99bb8bba529c
4,TCGA-60-2713,4afe2df4-c573-411a-ae7d-f6188e3591ae.mirbase21...,0bff265a-8c84-48d9-b40b-99fa252c9b2a
...,...,...,...
518,TCGA-85-8276,94c1b350-32bc-4b5e-be63-891a03bc6adc.mirbase21...,58ae470d-06ef-4be9-a3c4-d50ab6a524fe
519,TCGA-60-2711,91cf5329-d3cc-4437-8078-25ea1f9d4969.mirbase21...,08c04fb8-7c57-4d9b-a814-13b0c15fadf4
520,TCGA-66-2765,938831e1-a87a-4122-a1e6-1e141c1d503f.mirbase21...,e0df26e5-2344-4b69-b1f7-dcfff0171f3f
521,TCGA-94-A5I6,9184cef7-fd5b-4ac1-8732-f1fe3cb97622.mirbase21...,88dc0d6e-4e46-43c4-a189-97fc049c2a0d


In [74]:
mirna_fname = download_mirna_files(mirna_files_df, data_directory)

In [62]:
untar_study(mirna_fname, data_directory)

'attachment; filename=gdc_download_20211114_174720.011649.tar.gz'

In [93]:
files_df = mirna_files_df
file_name = mirna_fname
output_directory = data_directory

In [96]:
untar_study(file_name, output_directory, fmt="")

In [123]:
miRNA_IDs = set()
patient_dfs = {}
for i in range(len(files_df)):
    patient_id = files_df["cases.0.submitter_id"].iloc[i]
    foldername = files_df["id"].iloc[i]
    patient_fname = os.listdir(os.path.join(output_directory, foldername))[0]
    patient_df = pd.read_csv(os.path.join(output_directory, foldername, patient_fname), sep="\t")
    miRNA_IDs.update(patient_df["miRNA_ID"].tolist())
    patient_dfs[patient_id] = patient_df

In [186]:
miRNA_df = pd.DataFrame({"patient_id":[], **{k:[] for k in miRNA_IDs}}).set_index("patient_id")
i=0
for patient_id in patient_dfs:
    patient_df = patient_dfs[patient_id]
    transposed_patient_df = patient_df[["miRNA_ID","reads_per_million_miRNA_mapped"]].set_index("miRNA_ID").transpose()
    transposed_patient_df["patient_id"] = [patient_id]
    transposed_patient_df = transposed_patient_df.set_index("patient_id")
    miRNA_df[transposed_patient_df.columns] = transposed_patient_df
    if i>5:
        break
    i+=1
miRNA_df

,hsa-mir-182,hsa-mir-1270,hsa-mir-548a-2,hsa-mir-452,hsa-mir-188,hsa-mir-378b,hsa-mir-4671,hsa-mir-1910,hsa-mir-3619,hsa-mir-4302,...,hsa-mir-4647,hsa-mir-5579,hsa-mir-410,hsa-mir-329-2,hsa-mir-548g,hsa-mir-889,hsa-mir-196a-2,hsa-mir-3116-2,hsa-mir-7156,hsa-mir-548aa-2
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-22-1011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
